### Временный блокнот для оценки обученной модели

In [16]:
# Общие
import os
import torch
import yaml
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
from transformers import Trainer, TrainingArguments

from torch.utils.data import DataLoader
from datasets import load_dataset

from peft import LoraConfig, PeftModel, get_peft_model, prepare_model_for_kbit_training

# Для оценки
import re
import string
import numpy as np
import torch

from torch.cuda.amp import autocast

from tqdm import tqdm 

# Для логера
import logging
from datetime import datetime
from transformers import TrainerCallback

RANDOM_STATE = 42
torch.seed = RANDOM_STATE
DEVICE = "cuda:1" if torch.cuda.is_available() else "cpu"

In [17]:
with open(f'./config.yaml', 'r') as file:
    CONFIG = yaml.safe_load(file)
CONFIG

{'model': {'name': 'google/gemma-3-4b-it',
  'model_name_log': 'gemma-3-4b-it',
  'cache_dir': '../ft_v1/models_cache',
  'max_length': 512,
  'max_new_tokens': 32,
  'tokenizer': {'padding_size': 'left'}},
 'lora': {'r': 16,
  'lora_alpha': 32,
  'lora_dropout': 0.05,
  'target_modules': ['q_proj', 'o_proj', 'v_proj', 'k_proj']},
 'inference': {'temp': 0.7},
 'train': {'model_save_dir': './saved_models',
  'epochs': 3,
  'train_batch': 8,
  'val_batch': 8,
  'test_batch': 8,
  'grad_accum': 8,
  'eval_step': 5000,
  'save_step': 5000,
  'torch_empty_cache_steps': 8,
  'log_step': 500,
  'lr': '3e-4',
  'weight_decay': 0.01},
 'logs': {'dir': './logs_all'}}

In [18]:
#Пути до моделей
path2Gemma3_1B = '../ndora/cache_dir'
path2Gemma3_4B = '../ft_v1/models_cache/'
path2Llama31_8B = '../ft_v1/models_cache/'

model_name_Gemma_1 = 'google/gemma-3-1b-it'
model_name_Gemma_4 = 'google/gemma-3-4b-it'

model_name_llama_31 = 'meta-llama/Llama-3.1-8B-Instruct'

#Пути до датасетов
SQuAD_2_path = '../ft_v1/cdatasets/' # Путь до датасета SQuAD 2.0


datasetSQUAD2 = load_dataset("rajpurkar/squad_v2", cache_dir=SQuAD_2_path)

train_dataset = datasetSQUAD2['train']
val_dataset = datasetSQUAD2['validation']

print(f"Size train part: {len(train_dataset)}")
print(f"Size validation part: {len(val_dataset)}")

Using the latest cached version of the dataset since rajpurkar/squad_v2 couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'squad_v2' at ../ft_v1/cdatasets/rajpurkar___squad_v2/squad_v2/0.0.0/3ffb306f725f7d2ce8394bc1873b24868140c412 (last modified on Sat Jun  7 18:35:48 2025).


Size train part: 130319
Size validation part: 11873


In [19]:
#Общая инструкция для промпта
INSTRUCTION = 'You are a strict AI assistant designed to answer a question based on context. Answer verbatim from the context. Do not guess or formulate, answer as it is in the context. If there is no answer in the context, answer "No answer".'

# Промпт, будет использоваться при обучении и оценки модели
def prompt_template(example):
    global INSTRUCTION
    return f"{INSTRUCTION}\nContext:{example['Context']}\nQuestion:{example['Question']}\nAnswer:{example['Answer']}"

def prompt_template(context, question, answer):
    global INSTRUCTION
    return f"{INSTRUCTION}\nContext:{context}\nQuestion:{question}\nAnswer:{answer}"

def prompt_template_test(context, question):
    global INSTRUCTION
    return f"{INSTRUCTION}\nContext:{context}\nQuestion:{question}\nAnswer:"

In [20]:
def prompt_answer_ft(batch):
    """ Функция для формирования промпротов для train
    (ОСОБЕННОСТЬ: SQuAD формат)
    return (prompt, answer) - кортеж
    """
    answers = [
        item['text'][0] if item['text'] else ""
        for item in batch['answers']
    ]
    prompts = [
        prompt_template(context, question, answer) 
        for context, question, answer in zip(batch['context'], batch['question'], answers)
    ]
    return {
        "prompt": prompts, 
        "answer": answers
    }


def prompt_answer_fn_test(batch):
    """ Функция для формирования промпротов для test or val
    (ОСОБЕННОСТЬ: SQuAD формат)
    return (prompt, answer) - кортеж
    """
    answers = [
        item['text'][0] if item['text'] else ""
        for item in batch['answers']
    ]
    prompts = [
        prompt_template_test(context, question) 
        for context, question in zip(batch['context'], batch['question'])
    ]
    return {
        "prompt": prompts, 
        "answer": answers
    }

In [21]:
def tokenized_text(batch, tokenizer, answer_pattern = "Answer:", max_len_seq = 1024):
    """ Функция для токенизации промптов

    return: input_ids, attention_mask, labels, answers
    """
    prompts = batch['prompt']
    answers = batch['answer']
    
    encodings = tokenizer(
        prompts,
        return_tensors="pt",
        max_length=max_len_seq,
        truncation=True,
        padding="max_length"
    )
    
    #Для лейблов - клонируем тензор токенизированного промпта
    labels = encodings['input_ids'].clone()
    for i in range(len(prompts)):
        #Находим индекс слова в промпте, которое считается началом ответа для языковой модели
        answer_start = prompts[i].find(answer_pattern)
        
        #Определяем начало слова ответа в токенизированном формате (длинна тензора отличается от длинны промпта как строки)
        answer_start_id = len(prompts[i])
        if answer_start != -1:
            answer_start_id = tokenizer(prompts[i][:answer_start], return_tensors="pt")['input_ids'].size(1)

        #Сколько токенов паддинга
        padding_tokens = max_len_seq - encodings['attention_mask'][i].sum().item() 

        #Что не является ответом - устанавливаем -100 (defualt в PyTorch and HF)
        labels[i, :padding_tokens + answer_start_id] = -100

    encodings['labels'] = labels
    encodings['answer'] = answers
    return encodings # input_ids, attention_mask, labels, answers
    

def dataset_preprocess(dataset, tokenizer, answer_pattern = "Answer:", max_len_seq = 1024, eval=False):
    """Функция для преобразования датасета"""
    if eval:
        dataset = dataset.map(
            prompt_answer_fn_test, #Функция, которая применяется ко всем строкам
            batched=True,        #Использовать батчинг
            num_proc=1,         #Количество процессов
            remove_columns=dataset.column_names,  #Удаляем исходные колонки
        ) #prompt, answer - NO TOKENIZE
    else:
        dataset = dataset.map(
            prompt_answer_ft, #Функция, которая применяется ко всем строкам
            batched=True,        #Использовать батчинг
            num_proc=1,         #Количество процессов
            remove_columns=dataset.column_names,  #Удаляем исходные колонки
        ) #prompt, answer - NO TOKENIZE

    dataset = dataset.map(
        lambda x: tokenized_text(x, tokenizer, answer_pattern=answer_pattern, max_len_seq=max_len_seq), #Функция, которая применяется ко всем строкам, #Функция, которая применяется ко всем строкам
        batched=True,        #Использовать батчинг
        num_proc=1,         #Количество процессов
        remove_columns=dataset.column_names,  #Удаляем исходные колонки
    )
    return dataset #На данном этапе в датасетах - input_ids, attention_mask, labels, answer

In [22]:
def custom_collate_fn(batch):
    input_ids = [torch.tensor(item['input_ids']) for item in batch]
    attention_mask = [torch.tensor(item['attention_mask']) for item in batch]
    labels = [torch.tensor(item['labels']) for item in batch]
    answers = [item['answer'] for item in batch]
    return {"input_ids": torch.stack(input_ids),"attention_mask": torch.stack(attention_mask),"labels": torch.stack(labels),"answer": answers}

In [23]:
#### Конфигурация квантования

quant_config = BitsAndBytesConfig(
    load_in_8bit=True,                     # Загрузить модель в 8-битном формате
    bnb_8bit_quant_type="nf8",             # Тип 8-битного квантования (может быть nf8 или int8)
    bnb_8bit_compute_dtype=torch.bfloat16, # Тип данных для вычислений в 8-битном режиме
    bnb_8bit_use_double_quant=False        # Использовать ли двойное квантование
)

tokenizer = AutoTokenizer.from_pretrained(
    CONFIG['model']['name'], 
    cache_dir=CONFIG['model']['cache_dir']
)
tokenizer.pad_token_id = 0 #Устанавливаем токен для отступа (Используется для добавления до максимальной длинны)
tokenizer.padding_side = CONFIG['model']['tokenizer']['padding_size'] #Добавлять до максимальной длинны справа


model = AutoModelForCausalLM.from_pretrained(
    CONFIG['model']['name'], 
    cache_dir=CONFIG['model']['cache_dir'],
    torch_dtype=torch.bfloat16,
    quantization_config=quant_config,
    attn_implementation="eager",
    device_map="cuda:1" # Автоматическое распределение по доступным GPU
)
model.use_cache=True

#Когда испольуется квант конфиг - модель уже на DEVICE
# model.to(DEVICE)

Unused kwargs: ['bnb_8bit_quant_type', 'bnb_8bit_compute_dtype', 'bnb_8bit_use_double_quant']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
loading file tokenizer.model from cache at ../ft_v1/models_cache/models--google--gemma-3-4b-it/snapshots/093f9f388b31de276ce2de164bdc2081324b9767/tokenizer.model
loading file tokenizer.json from cache at ../ft_v1/models_cache/models--google--gemma-3-4b-it/snapshots/093f9f388b31de276ce2de164bdc2081324b9767/tokenizer.json
loading file added_tokens.json from cache at ../ft_v1/models_cache/models--google--gemma-3-4b-it/snapshots/093f9f388b31de276ce2de164bdc2081324b9767/added_tokens.json
loading file special_tokens_map.json from cache at ../ft_v1/models_cache/models--google--gemma-3-4b-it/snapshots/093f9f388b31de276ce2de164bdc2081324b9767/special_tokens_map.json
loading file tokenizer_config.json from cache at ../ft_v1/models_cache/models--google--gemma-3-4b-it/snapshots/093f9f388b31de276ce2de164bdc20

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

All model checkpoint weights were used when initializing Gemma3ForConditionalGeneration.

All the weights of Gemma3ForConditionalGeneration were initialized from the model checkpoint at google/gemma-3-4b-it.
If your task is similar to the task the model of the checkpoint was trained on, you can already use Gemma3ForConditionalGeneration for predictions without further training.
loading configuration file generation_config.json from cache at ../ft_v1/models_cache/models--google--gemma-3-4b-it/snapshots/093f9f388b31de276ce2de164bdc2081324b9767/generation_config.json
Generate config GenerationConfig {
  "bos_token_id": 2,
  "cache_implementation": "hybrid",
  "do_sample": true,
  "eos_token_id": [
    1,
    106
  ],
  "pad_token_id": 0,
  "top_k": 64,
  "top_p": 0.95
}



In [24]:
# Создать даталоадер
val_dataloader = DataLoader(
    val_dataset,                # Датасет (например, tokenized_dataset)
    batch_size=CONFIG['train']['val_batch'],         # Размер батча
    shuffle=False,         # Перемешивать данные
    num_workers=0,         # Количество потоков для загрузки
    collate_fn=custom_collate_fn,       # Функция для сборки батча
    pin_memory=False,      # Копировать данные в CUDA-память
    drop_last=False,       # Отбрасывать последний неполный батч
    prefetch_factor=None,     # Количество батчей для предварительной загрузки (сколько батчей будет загружено сразу для  ускорения, только в параллельном режиме (когда num_workers > 0)) (None - если не используем)
    persistent_workers=False  # Сохранять рабочие потоки между итерациями
)

print(f"size val dataloader: {len(val_dataloader)}")

size val dataloader: 1485


In [25]:
def normalize_text(text):
    """
    Нормализует текст: нижний регистр, удаление пробелов и пунктуации
    """
    text = text.lower() # Приводим к нижнему регистру
    text = text.translate(str.maketrans("", "", string.punctuation)) # Удаляем пунктуацию
    text = re.sub(r'\s+', ' ', text).strip() # Удаляем лишние пробелы
    return text

def compute_exact_match(prediction, ground_truth):
    """
    Вычисляет Exact Match для одного примера
    Принимает 2 строки для сравнения
    """
    return int(normalize_text(prediction) == normalize_text(ground_truth))

def compute_f1_score(prediction, ground_truth):
    """
    Вычисляет F1 Score для одного примера
    Принимает 2 строки для сравнения
    """
    pred_tokens = normalize_text(prediction).split()
    truth_tokens = normalize_text(ground_truth).split()
    
    if len(pred_tokens) == 0 and len(truth_tokens) == 0: # Если оба ответа пустые, F1 = 1
        return 1.0
    if len(pred_tokens) == 0 or len(truth_tokens) == 0: # Если один из ответов пустой, F1 = 0
        return 0.0
    
    # Находим общие токены
    common_tokens = set(pred_tokens) & set(truth_tokens)
    tp = len(common_tokens)
    
    precision = tp / len(pred_tokens)  # Precision = TP / (TP + FP), где FP = предсказанные токены, не входящие в правильные
    recall = tp / len(truth_tokens) # Recall = TP / (TP + FN), где FN = правильные токены, не входящие в предсказанные
    
    # F1 = 2 * (precision * recall) / (precision + recall)
    if precision + recall == 0:
        return 0.0
    return 2 * (precision * recall) / (precision + recall)


def calculate_evaluate_metrics_EM_F1(em_all, f1_all):
    ''' 
    Функция для вычисления среднего значения EM и F1
    '''
    em_all = np.array(em_all)
    f1_all = np.array(f1_all)
    print(f"EM: {em_all.mean()}, F1: {f1_all.mean()}")
    return em_all.mean(), f1_all.mean()


def evaluate_model_for_metrics(model, tokenizer, dataloader, device, max_new_tokens=32, temp=0.7, logger=None, pfraze_no_answ="No answer"):
    """Функция для оценки модели по метриками EM, F1 """
    em_all = []
    f1_all = []

    if logger:
        logger.log("Test model. Calculate metrics")

    for batch, item in enumerate(tqdm(dataloader, desc="Test model")):
        with autocast(dtype=torch.bfloat16): #Использовать смешанную точность (Работает)
            #Генерируем батч
            pred = model.generate(
                input_ids=item['input_ids'].to(device),
                attention_mask=item['attention_mask'].to(device),
                max_new_tokens=max_new_tokens,
                temperature=temp
            )

            tensor_shape_1 = item['input_ids'].shape[1]
            answers_model_text = tokenizer.batch_decode(pred[:, tensor_shape_1:], skip_special_tokens=True)
            # item['answer']

            #Лишний цикл, но если внести в циклы которые ниже, то ничего не вычисляет
            #только с помощью этого стало нормально определять, если 
            # answers_model_text = ["" if normalize_text(item) == "no answer" else item for item in answers_model_text]
            
            tmp_em = [compute_exact_match("", answer)
                      if normalize_text(predict) == "no answer" else compute_exact_match(predict, answer)
                      for predict, answer in zip(answers_model_text, item['answer'])
            ]
            tmp_f1 = [compute_f1_score("", answer)
                      if normalize_text(predict) == "no answer" else compute_f1_score(predict, answer)
                      for predict, answer in zip(answers_model_text, item['answer'])
             ]
            
            if logger:
                for predict, answer, emtmp, f1tmp in zip(answers_model_text, item['answer'], tmp_em, tmp_f1):
                    logger.log(f"\nPRED:{predict}\nANSW:{answer}\nEM:{emtmp}\nF1:{f1tmp}\n")
        
        em_all += tmp_em
        f1_all += tmp_f1
    
    em, f1 = calculate_evaluate_metrics_EM_F1(em_all, f1_all)
    if logger:
        logger.log(f"em_all_len: {len(em_all)} f1_all_len: {len(f1_all)}")
        logger.log(f"EM: {em} F1: {f1}")
    return em_all, f1_all, em, f1


In [26]:
# Для отлова логгирования метрик
class CustomLoggingCallback(TrainerCallback):
    def __init__(self, logger):
        self.logger = logger

    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs is not None:
            # Записываем метрики в лог
            self.logger.info(
            "\n" + "\n".join([f"Step {state.global_step}: {key} = {value}" for key, value in logs.items()]))


class CustomLogger:
    def __init__(self, log_dir):
        self._timestamp = datetime.now().strftime("%d-%m-%Y_%H-%M-%S")
        self._log_filename = f"{log_dir}/log_run_{self._timestamp}.log"
        
        logging.basicConfig(
            level=logging.INFO,  # Уровень логирования
            format='%(asctime)s - %(levelname)s - %(message)s',  # Формат сообщений
            handlers=[
                logging.FileHandler(self._log_filename),  # Запись в файл
                # logging.StreamHandler()  # Вывод в консоль (опционально)
            ],
            encoding='UTF-8'
        )

        transformers_logger = logging.getLogger("transformers")
        transformers_logger.setLevel(logging.INFO)
        file_handler = logging.FileHandler(self._log_filename)
        file_handler.setFormatter(logging.Formatter('%(asctime)s - %(levelname)s - %(message)s'))
        transformers_logger.addHandler(file_handler)

        self._my_logger = logging.getLogger("my_custom_logger") #Перехватить логгер transformers
        self._my_logger.setLevel(logging.INFO)


    def log(self, text_for_log : str):
        self._my_logger.info(text_for_log)

In [27]:
time_stamp = datetime.now().strftime("%d-%m-%Y_%H-%M-%S")

PREFIX_PATH_FOR_MODEL_DIRS = f'{CONFIG['logs']['dir']}/train__{time_stamp}__{CONFIG['model']['model_name_log']}'

os.makedirs(PREFIX_PATH_FOR_MODEL_DIRS, exist_ok=True)
mylogger = CustomLogger(log_dir=PREFIX_PATH_FOR_MODEL_DIRS)

mylogger.log(f"""Start logger
--- CONFIGURATE --- 
{CONFIG}
---  --- 
""")

### Загрузка LoRA конфига, который использовался при обучении

In [28]:
# checkpoint обучения
check_path = "./saved_models/save_state_train_08-06-2025_00-20-25_0.1105502/"

In [29]:
# lora_config = PeftConfig.from_pretrained(check_path)

model = PeftModel.from_pretrained(model, check_path)

model = prepare_model_for_kbit_training(model)

# model = get_peft_model(model, lora_config)
mylogger.log(f"""
model architecture:
{str(model)}

model congif:
{model.config}

training parameters:
{model.print_trainable_parameters()}
""")
print(f"\nLoRA model:\n{str(model)}")

RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
em_all, f1_all, em, f1 = evaluate_model_for_metrics(
    model,
    tokenizer,
    val_dataloader,
    DEVICE,
    max_new_tokens=CONFIG['model']['max_new_tokens'],
    logger=mylogger,
    pfraze_no_answ="No answer"
)

In [ ]:
### Оценка тонко настроенной модели:
### Какое сохранение использовал, путь:


